<a href="https://colab.research.google.com/github/pedrecho/scrap_news/blob/main/citaty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

In [6]:
url = 'https://citaty.info/man?page='
sites = ''
for i in range(1):
  sites += requests.get(url + str(i)).text

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(sites, "html.parser")

In [44]:
quotes = soup.findAll('article')
print(len(quotes))

16


In [51]:
text = []
author = []
tags = []
rating = []
for item in quotes:
  text.append(item.div.div.div.div.p.text.replace('\xa0', ' '))
  author.append(item.find('div', class_='field field-name-taxonomy-vocabulary-3 field-type-taxonomy-term-reference field-label-hidden').div.div.a.text)
  tg = item.find('div', class_='node__topics')
  tags.append([])
  if tg is not None:
    tg = tg.findAll('div', class_='field-items')
    for i in tg:
      tags[-1] += list(map(lambda x: x.text, i.findAll('a')))
  rating.append(int(item.find('div', class_='rating__value__digits').text.strip('\n ')))


In [52]:
import pandas as pd

In [53]:
data = pd.DataFrame({
    'Text': text,
    'Author': author,
    'Tags': tags,
    'Rating': rating
})
data.head()

,Text,Author,Tags,Rating
0,"Мир полон волшебных вещей, терпеливо ожидающих...",Уильям Батлер Йейтс,"[мир, мироздание]",10
1,Живая жизнь не может быть определена никаким к...,Викентий Викентьевич Вересаев,"[мудрые цитаты, жизнь, добро, борьба, смысл жи...",19
2,"У всех у нас есть страхи. Но у тех, кто смотри...",Эрнест Хемингуэй,"[цитаты со смыслом, смелость, храбрость, страх]",23
3,"Не тот твой друг, кто за столом с тобою пьёт,\...",Омар Хайям,"[мудрые цитаты, друзья, дружба, верность]",11
4,​​Самые прекрасные мгновения всегда полны грус...,Андре Моруа,"[жизненные цитаты, печаль, мгновение, момент]",14
